In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data3.json", "r", encoding="utf8"))


In [3]:
texts_len = df['text'].apply(len)
df.drop(df[texts_len<50].index, inplace=True)

In [4]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [5]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['themes'].to_list())

In [6]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [7]:
embedding_dim = 64

model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(500, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(len(Y[0]), activation='sigmoid')
])

model1.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5

In [8]:
epochs = 5
model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs)

Epoch 1/5
208/208 [==============================] - 60s 287ms/step - loss: 1.4397 - categorical_accuracy: 0.4411 - val_loss: 1.3720 - val_categorical_accuracy: 0.4432
Epoch 2/5
208/208 [==============================] - 61s 291ms/step - loss: 1.0540 - categorical_accuracy: 0.6630 - val_loss: 1.4765 - val_categorical_accuracy: 0.4851
Epoch 3/5
208/208 [==============================] - 55s 265ms/step - loss: 0.7617 - categorical_accuracy: 0.8125 - val_loss: 1.8200 - val_categorical_accuracy: 0.4811
Epoch 4/5
208/208 [==============================] - 49s 235ms/step - loss: 0.6548 - categorical_accuracy: 0.8187 - val_loss: 2.0236 - val_categorical_accuracy: 0.5014
Epoch 5/5
208/208 [==============================] - 49s 235ms/step - loss: 0.6172 - categorical_accuracy: 0.8252 - val_loss: 2.2951 - val_categorical_accuracy: 0.4973


In [9]:
score = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

24/24 [==============================] - 0s 16ms/step - loss: 2.2951 - categorical_accuracy: 0.4973
Test Score: 2.295149087905884
Test Accuracy: 0.4972972869873047


In [10]:

'''y_pred = model1.predict(X_test)
class_names = ['in', 'out']
cm = confusion_matrix(Y_test, np.rint(y_pred))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot()
plt.show()'''

"y_pred = model1.predict(X_test)\nclass_names = ['in', 'out']\ncm = confusion_matrix(Y_test, np.rint(y_pred))\ndisp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)\ndisp.plot()\nplt.show()"

### Model 2 LSTM

In [11]:
embedding_dim =100 

model2 = keras.Sequential([
    layers.Embedding(max_features, embedding_dim, input_length=max_len),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(len(Y[0]), activation='sigmoid')
])

In [12]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 1,080,703
Trainable params: 1,080,703
Non-trainable params: 0
_________________________________________________________________


In [14]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string
batch_size = 128
epochs = 5

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[early_stopping_callback])

Epoch 1/5
47/47 [==============================] - 13s 287ms/step - loss: 1.4935 - categorical_accuracy: 0.4318 - val_loss: 1.4781 - val_categorical_accuracy: 0.4339
Epoch 2/5
47/47 [==============================] - 24s 512ms/step - loss: 1.4788 - categorical_accuracy: 0.4516 - val_loss: 1.4845 - val_categorical_accuracy: 0.4399
Epoch 3/5
47/47 [==============================] - 10s 211ms/step - loss: 1.4565 - categorical_accuracy: 0.4759 - val_loss: 1.4786 - val_categorical_accuracy: 0.4339
Epoch 4/5
47/47 [==============================] - 20s 435ms/step - loss: 1.4165 - categorical_accuracy: 0.5035 - val_loss: 1.4828 - val_categorical_accuracy: 0.4354


In [15]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

24/24 [==============================] - 1s 39ms/step - loss: 1.5005 - categorical_accuracy: 0.4324
Test set
  Loss: 1.501
  Accuracy: 0.432


In [16]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [18]:
batch_size = 64
epochs = 5
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
94/94 [==============================] - 6s 62ms/step - loss: 558.4261 - accuracy: 0.3613 - val_loss: 1755.3889 - val_accuracy: 0.4339
Epoch 2/5
94/94 [==============================] - 6s 60ms/step - loss: 40645.6367 - accuracy: 0.3665 - val_loss: 59584.5703 - val_accuracy: 0.4339
Epoch 3/5
94/94 [==============================] - 5s 58ms/step - loss: 231425.7500 - accuracy: 0.3638 - val_loss: 87065.5625 - val_accuracy: 0.1592
Epoch 4/5
94/94 [==============================] - 6s 61ms/step - loss: 225192.7500 - accuracy: 0.3602 - val_loss: 335396.1562 - val_accuracy: 0.1592
Epoch 5/5
94/94 [==============================] - 5s 58ms/step - loss: 465405.6875 - accuracy: 0.3568 - val_loss: 637336.1875 - val_accuracy: 0.4339


In [19]:
accr = model3.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

24/24 [==============================] - 0s 8ms/step - loss: 691659.3125 - accuracy: 0.3973
Test set
  Loss: 691659.312
  Accuracy: 0.397


In [20]:
len(df.themes[0])

KeyError: 0